In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from paik.solver import Solver, DEFAULT_SOLVER_PARAM_M7_NORM

solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7_NORM)

num_im = 100
num_ta = 100
num_sols = 100

solver.shrink_ratio = 0.0
J_im, P_im, F_im = solver.random_sample_JPF(num_samples=num_im)
J_ta, P_ta, F_ta = solver.random_sample_JPF(num_samples=num_ta)
knn = NearestNeighbors(n_neighbors=1).fit(np.column_stack((solver._J_tr, solver._P_tr)))


def get_F(J, P):
    J, P = np.atleast_2d(J), np.atleast_2d(P)
    _, idx = knn.kneighbors(np.column_stack((J, P)))
    return solver._F[idx.flatten()]

In [ ]:
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, (j_im, f_im) in enumerate(zip(J_im, F_im)):
    # F = np.full_like(F_ta, *f_im)
    j_im = np.tile(j_im, (num_ta, 1))
    F = get_F(j_im, P_ta)
    J = solver.solve(P=P_ta, F=F, num_sols=num_sols, return_numpy=True)
    l2_errs[i], ang_errs[i] = solver.evaluate_solutions(J, P_ta, return_col=True)
    config_errs[i] = np.linalg.norm(J - j_im, axis=2).mean(axis=0)

In [ ]:
df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": np.rad2deg(ang_errs.flatten()),
        "config_err": np.rad2deg(config_errs.flatten()),
    }
)

df.describe()

In [ ]:
import numpy as np
import pandas as pd
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

# -> unrefined solutions
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, p in enumerate(P_ta):
    (
        solutions,
        l2_errors,
        angular_errors,
        _,
        _,
        _,
    ) = ik_solver.solve(
        p, n=num_im, refine_solutions=False, return_detailed=True
    )  # type: ignore

    solutions = solutions.detach().cpu().numpy()

    l2_errs[:, i] = l2_errors
    ang_errs[:, i] = angular_errors

    for j, j_im in enumerate(J_im):
        config_errs[j, i] = np.linalg.norm(solutions - j_im, axis=1).mean()

In [ ]:
df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": np.rad2deg(ang_errs.flatten()),
        "config_err": np.rad2deg(config_errs.flatten()),
    }
)

df.describe()